In [1]:
import os
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\User\\Desktop\\MLOps Project\\End-to-End-ML-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 9601-F3BC

 Directory of c:\Users\User\Desktop\MLOps Project\End-to-End-ML-Project

01/28/2024  09:43 PM    <DIR>          .
01/28/2024  09:43 PM    <DIR>          ..
01/28/2024  11:59 AM             3,238 .gitignore
01/28/2024  12:55 PM                 0 app.py
01/30/2024  02:01 PM    <DIR>          artifacts
01/28/2024  12:55 PM    <DIR>          config
01/30/2024  02:17 PM    <DIR>          Data
01/28/2024  11:59 AM             1,089 LICENSE
01/28/2024  04:33 PM    <DIR>          logs
01/28/2024  09:43 PM               881 main.py
01/30/2024  02:01 PM                44 params.yaml
01/30/2024  05:41 PM               517 README.md
01/28/2024  01:19 PM               138 requirements.txt
01/30/2024  01:31 PM    <DIR>          research
01/28/2024  07:48 PM               378 schema.yaml
01/28/2024  07:20 PM               771 setup.py
01/30/2024  06:37 PM    <DIR>          src
01/28/2024  12:55 PM             1,266 template.py
01/28

In [6]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from MLProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-30 18:39:14,539: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-30 18:39:14,541: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-30 18:39:14,544: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-30 18:39:14,545: INFO: common: created directory at: artifacts]
[2024-01-30 18:39:14,546: INFO: common: created directory at: artifacts/model_trainer]


ValueError: could not convert string to float: 'white'